# EdgeR CABG vs Control
## Analysis date: 2022/03/02

In [1]:
library(edgeR)
library(magrittr)
library(ggplot2)
library(DESeq2)

Loading required package: limma

Warning message:
“package ‘ggplot2’ was built under R version 4.0.5”
Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from ‘package:limma’:

    plotMA


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position,

In [2]:
INDIR <- '.'
OUTDIR <- '.'

In [3]:
if (!file.exists(OUTDIR)){
    dir.create(file.path(OUTDIR))
}

## Cell_type level

In [4]:
x=read.csv(paste(INDIR, "ALL_CELLTYPE_PSEUDOBULK.csv", sep='/'), row.names=1)
genes_tofilter=read.csv(paste(INDIR, "ALL_CELLTYPE_PSEUDOBULK_FILTERING.csv", sep='/'))

# Only needed for the column cell_state
#CELLTYPE_STATE <- read.csv(paste(INDIR, "CELLSTATE_CELLTYPE_MAP.csv", sep='/'))
#colnames(CELLTYPE_STATE) <- c("cell_states", 'cell_type')

# Removes patients with only few (<10) nuclei numbers (as they might bias the analysis)
CELLTYPE_FILTER <- read.csv(paste(INDIR, "ABSOLUTE_CELLTYPE_NUMBER.csv", sep='/'))

#rownames(x) <- x$X
#x <- x[,-1]
l <- strsplit(colnames(x), "__")

In [6]:
meta.data <- t(as.data.frame(l))
rownames(meta.data) <- colnames(x)

meta.data <- as.data.frame(meta.data)
colnames(meta.data)  <-  c("Group", 'Patient', 'cell_type')

In [10]:
meta.data$Patient_Group <- paste(meta.data$Patient, meta.data$Group, sep="_")

In [11]:
as.character(unique(meta.data$cell_type))

[1] "EC"       "FB"       "Lymphoid" "Myeloid"  "Neuronal" "PC"      
 [7] "SMC"      "vCM"      "Mast"     "AD"

In [12]:
GROUPS <- c('IHD')
CELL_LEVEL <- 'CELLTYPE'

In [14]:
i <- 1

for(GROUP in as.character(GROUPS)){
    message("\n###START: ", GROUP," #####\n")
    
    #for(CELL_STATE in as.character(unique(meta.data$cell_type))){
    for(CELL_STATE in c('EC','FB','PC','vCM','SMC')){
         
        CONTROL_COLUMN <- paste("control__", sep="")
        GROUP_COLUMN <- paste(GROUP,"__", sep="")
        #CELLTYPE <- as.character(CELLTYPE_STATE[which(CELLTYPE_STATE$cell_type==CELL_STATE),"cell_type"])
        #CELL_STATE <- 'BULK'
        CELLTYPE <- CELL_STATE
        
        x_sub <- x[,which(meta.data$Group %in% c("control", GROUP))]
        meta.data_sub <- meta.data[which(meta.data$Group %in% c("control", GROUP)),]
        
        x_sub <- x_sub[,which(meta.data_sub$cell_type %in% c(CELL_STATE))]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$cell_type %in% c(CELL_STATE)),]
        
        #### remove first line (cell_type column)
        CELLTYPE_FILTER_SUB <- CELLTYPE_FILTER
        PATIENTS_TOKEEP <- CELLTYPE_FILTER_SUB[which(CELLTYPE_FILTER_SUB$cell_type==CELL_STATE),-1]
        PATIENTS_TOKEEP <- colnames(PATIENTS_TOKEEP)[which(as.logical(PATIENTS_TOKEEP))]
        PATIENTS_TOKEEP <- gsub('^X','',PATIENTS_TOKEEP)
        #if (length(PATIENTS_TOKEEP) <= 1) {
        #    next
        #}
        ####
        
        # Subset only for patients which fulfill condition
        x_sub <- x_sub[,which(meta.data_sub$Patient %in% PATIENTS_TOKEEP)]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$Patient %in% PATIENTS_TOKEEP),]
        #meta.data_sub$Group = droplevels(meta.data_sub$Group) ## Drop factor levels
   
        
        # The test is only run, if 2 conditions have at least 2 patients
        if (length(unique(meta.data_sub$Group))==2 & 
            all(table(as.character(meta.data_sub$Group))>=3))
           {
             # Prepare DGEList object
            meta.data_sub$Group <- as.factor(as.character(meta.data_sub$Group))
            dge <- DGEList(counts=x_sub, group=meta.data_sub$Group)
            
            # Filter genes based on expression, the qlf Object will be filtered to exclude them from FDR calculation
            genes_tofilter_sub_GROUP <- genes_tofilter[,which(gsub("^(.*)__.*__.*$", "\\1", colnames(genes_tofilter)) %in% GROUP &
                                                             gsub("^.*__(.*)__.*$", "\\1", colnames(genes_tofilter)) %in% PATIENTS_TOKEEP &
                                                             gsub("^.*__.*__(.*)$", "\\1", colnames(genes_tofilter)) %in% CELL_STATE)]

            genes_tofilter_sub_CONTROL <- genes_tofilter[,which(gsub("^(.*)__.*__.*$", "\\1", colnames(genes_tofilter)) %in% 'control' &
                                                             gsub("^.*__(.*)__.*$", "\\1", colnames(genes_tofilter)) %in% PATIENTS_TOKEEP &
                                                             gsub("^.*__.*__(.*)$", "\\1", colnames(genes_tofilter)) %in% CELL_STATE)]
            
            genes_tofilter_sub <- data.frame("X"=genes_tofilter[,"gene_name"],
                                             CONTROL_COLUMN=apply(genes_tofilter_sub_CONTROL, 1, mean),
                                             GROUP_COLUMN=apply(genes_tofilter_sub_GROUP, 1, mean))
            colnames(genes_tofilter_sub) <- c("X", CONTROL_COLUMN, GROUP_COLUMN)
            
            keep <- which(genes_tofilter_sub[,CONTROL_COLUMN] > 0.0125 |
                          genes_tofilter_sub[,GROUP_COLUMN] > 0.0125)
            
            #keep <- genes_tofilter[,CONTROL_COLUMN]>0.0125 | ##### change & to | ##### 
            #        genes_tofilter[,REGION_COLUMN]>0.0125
            #dge <- dge[keep, , keep.lib.sizes=FALSE]
    
            # PP, model matrix, https://www.nature.com/articles/nmeth.4612 (edgeRQLFDetRate)
            dge <- calcNormFactors(dge)
            cdr <- scale(colMeans(x_sub > 0))
            design <- model.matrix(~ cdr + meta.data_sub$Group)
    
            # 
            dge <- estimateDisp(dge, design = design)
            fit <- glmQLFit(dge, design = design)
            qlf <- glmQLFTest(fit)
    
            # For all
            tt <- topTags(qlf, n = Inf)
            
            # Only for "expressed/detected" genes
            tt_filtered <- topTags(qlf[keep,], n = Inf)
            
            tt$table[,"Gene"] <- as.character(rownames(tt$table))
            tt_filtered$table[,"Gene"] <- as.character(rownames(tt_filtered$table))
            tt_filtered$table[,"low_expression"] <- "F" #####  Low-expression column ##### 
            tt_filtered$table[,"FDR_plot"] <- -log10(tt_filtered$table$FDR) #####  For plotting Volcano, here we use FDR, not pValue  ##### 
            
            
            tt_merged <- merge(tt$table[,c("Gene", "logFC", "logCPM", 'F', 'PValue')], 
                               tt_filtered$table[,c("Gene", "FDR", "low_expression", "FDR_plot")], 
                               by="Gene", all=T)   
            
            # Replace NA FDRs with NA                   #####  ORDER CHANGED #####  
            tt_merged[which(is.na(tt_merged$FDR)),"FDR"] <- 1 
            
            EXPRESSION_MEAN <- genes_tofilter_sub[,c("X", CONTROL_COLUMN, GROUP_COLUMN)]
            colnames(EXPRESSION_MEAN) <- c("Gene", "mean_exp_Control", 'mean_exp_disease')
            tt_merged <- merge(tt_merged, EXPRESSION_MEAN, by="Gene")
    
            
            tt_merged[,"Group"] <- GROUP
            tt_merged[,"annotation_level"] <- CELL_LEVEL
            tt_merged[,"cell_state"] <- CELL_STATE
            tt_merged[,"cell_type"] <- CELLTYPE
            tt_merged[,"comparison"] <- paste("Control_", GROUP, sep="")
            
            ###### Suggestion to add those columns
            tt_merged[,"Observations_group"] <- as.numeric(table(meta.data_sub$Group)[GROUP])
            tt_merged[,"Observations_reference"] <- as.numeric(table(meta.data_sub$Group)['control'])
            
            # low expression add T
            tt_merged[which(is.na(tt_merged$low_expression)),"low_expression"] <- "T"
            
            if(i==1){
                 final_df <- tt_merged 
                 dge_cpm <- cpm(dge,normalized.lib.sizes=FALSE)
                 sn_mean_exp <- data.frame(genes_tofilter_sub_GROUP, genes_tofilter_sub_CONTROL, row.names=genes_tofilter[,"gene_name"])
            } else {
                final_df <- rbind(final_df, tt_merged)
                dge_cpm <- merge(dge_cpm, cpm(dge,normalized.lib.sizes=FALSE), by=0, all=TRUE) 
                rownames(dge_cpm) <- dge_cpm$Row.names
                dge_cpm <- dge_cpm[,-1]
                sn_mean_exp <- data.frame(sn_mean_exp, genes_tofilter_sub_GROUP, genes_tofilter_sub_CONTROL, row.names=genes_tofilter[,"gene_name"])
            }
            
            i <- i + 1
            message("\n###FINISHED: ", CELL_STATE," #####\n")
        } else {
            message("\n###Skip: ", CELL_STATE," #####\n")
        }
    }
}


###START: IHD #####



###FINISHED: EC #####



###FINISHED: FB #####



###FINISHED: PC #####



###FINISHED: vCM #####



###FINISHED: SMC #####




In [19]:
write.csv(final_df, paste(OUTDIR,"ALL_CELLTYPE_Control_vs_CABG_EDGER.csv", sep='/'))

In [21]:
#dge_cpm <- cpm(dge)
write.csv(dge_cpm, paste(OUTDIR,"ALL_CELLTYPE_Control_vs_CABG_CPMs.csv", sep='/'))

In [22]:
write.csv(sn_mean_exp, paste(OUTDIR,"ALL_CELLTYPE_snRNA_MEAN_EXPRESSION_FILTERED_CABG_combined.csv", sep='/'))